# Aula 5: SQL

## Introdução

Nesta aula veremos as principais queries de consulta em sql

## Conexao no db4free via sqlalchemy

In [1]:
import sqlite3
import pandas as pd
from pandasql import sqldf

![](https://ensinandomaquinasblog.files.wordpress.com/2020/11/f1.png?w=1024)
![](https://i.imgur.com/Jory0O3.png)

In [2]:
con = sqlite3.connect('salesOlist.db')
cursor = con.cursor()
cursor.execute('SELECT name FROM sqlite_master WHERE type="table" ORDER BY name')
cursor.fetchall()

[('ClosedDeals',),
 ('Customers',),
 ('Geolocation',),
 ('OrderItems',),
 ('OrderPayments',),
 ('OrderReviews',),
 ('Orders',),
 ('Products',),
 ('QualifiedLeads',),
 ('Sellers',)]

## Subqueries

- Uma Subquery (também conhecida como SUBCONSULTA ou SUBSELECT) é uma instrução do tipo SELECT dentro de outra instrução SQL
- Desta forma, se torna possível efetuar consultas que de outra forma seriam extremamente complicadas ou impossíveis de serem feitas de outra forma

`SELECT
    *
FROM
    tabela1 AS T
WHERE
    coluna1 IN
    (
        SELECT
            coluna2
        FROM
            tabela2 AS T2
        WHERE
            T.id = T2.id
    )`

## Como utilizar?

Existem algumas formas de se utilizar subqueries

- Subquery como uma nova coluna da consulta `(SELECT AS FIELD)`
- Subquery como filtro de uma consulta (utilizando `IN, EXISTS` ou operadores de comparação).
- Subquery como fonte de dados de uma consulta principal (`SELECT FROM SELECT`)

### Subquery como filtro de uma nova consulta

- Podemos utilizar subqueries fazendo filtros no resultado de outras consultas
- Para esse modelo podemos utilizar as cláusulas IN ou operadores de comparação, como =, >=, <=, dentre outros.

In [12]:
query = '''
    SELECT order_id
            ,order_purchase_timestamp
            ,customer_id
    FROM Orders
    WHERE customer_id IN (
        SELECT customer_id
        FROM Customers
        WHERE customer_state = 'SP'
    )
    ORDER BY order_purchase_timestamp DESC
'''

pd.read_sql(query, con)

,order_id,order_purchase_timestamp,customer_id
0,10a045cdf6a5650c21e9cfeb60384c16,2018-10-17 17:30:18,a4b417188addbc05b26b72d5e44837a1
1,a2ac6dad85cf8af5b0afb510a240fe8c,2018-10-03 18:55:29,4c2ec60c29d10c34bd49cb88aa85cfc4
2,392ed9afd714e3c74767d0c4d3e3f477,2018-09-29 09:13:03,2823ffda607a2316375088e0d00005ec
3,87ae60ef8b08ae0e5f903cacb53a6904,2018-09-11 16:45:54,fea4d57ed3a45455f89c25ef3dae8ee8
4,a89abace0dcc01eeb267a9660b5ac126,2018-09-06 18:45:47,2f0524a7b1b3845a1a57fcf3910c4333
...,...,...,...
41741,d207cc272675637bfed0062edffd0818,2016-10-03 22:06:03,b8cf418e97ae795672d326288dfab7a7
41742,3b697a20d9e427646d92567910af6d57,2016-10-03 09:44:50,355077684019f7f60a031656bd7262b8
41743,71303d7e93b399f5bcd537d124c0bcfa,2016-10-02 22:07:52,b106b360fe2ef8849fbbd056f777b4d5
41744,bfbd0f9bdef84302105ad712db648a6c,2016-09-15 12:16:38,86dc2ffce2dfff336de2f386a786e574


### Subquery como uma nova coluna da consulta

Uma das formas possíveis de se realizar uma subquery é fazendo com que o resultado de outra consulta seja uma coluna dentro da sua consulta principal

In [13]:
query = '''
    SELECT order_id
            ,order_purchase_timestamp
            ,(
                SELECT MAX(price)
                FROM OrderItems AS a
                WHERE a.order_id = b.order_id
            ) AS max_price
    FROM Orders AS b 
    WHERE b.order_status = 'shipped'  
    ORDER BY max_price DESC
'''

pd.read_sql(query, con)

,order_id,order_purchase_timestamp,max_price
0,73ef343c9dbf2f6d45362c9bc150f4b7,2018-08-12 03:44:50,2150.00
1,f0a2783007ba2c431132d38c7642d5c9,2018-04-17 11:10:00,2110.00
2,2cc93fd6f68e6cfcb968814de8a14296,2017-10-11 18:25:49,1989.00
3,9562a9640c9bea4e43a21cac378d8781,2017-05-06 19:58:48,1697.00
4,b534c927d5cb15838b514c99f1c8d94d,2017-09-29 20:51:26,1599.99
...,...,...,...
1102,6c37458893de8ddaa38b557e58e3bfb7,2018-01-06 15:42:24,6.99
1103,f8a0c2f75253cacddb95753652794c0f,2017-04-06 23:05:31,5.90
1104,e723fd90e2d095c2762e5e800d759bfa,2017-11-25 17:14:25,5.00
1105,d642656598ae928a250620315d19e87e,2018-03-07 10:43:39,4.99


### Subquery como fonte de dados de uma consulta principal

Este outro formato faz com que o resultado de uma subquery possa ser utilizado como tabela fonte de dados de uma consulta principal

In [36]:
query = '''
    SELECT AVG(order_count) avg_order_count
    FROM
    (
        SELECT seller_id
                ,COUNT(order_id) order_count
        FROM OrderItems
        GROUP BY seller_id
    ) t
    

'''

pd.read_sql(query, con)

,avg_order_count
0,36.397415
